In [1]:
from PIL import Image
import numpy as np
from skimage import transform
import os
import shutil
import tensorflow as tf
from tensorflow import keras
import cv2
import time


def load(image):
      
    np_image = np.array(image).astype('float32')/255
    np_image = transform.resize(np_image, (128,128, 3))
    np_image = np.expand_dims(np_image, axis=0)
    return np_image

def predict_result(image,threshold):
    outputs = model.predict(image)  
    
    s = outputs[0][0]
    if s >= threshold:  
        status = 1
        return s, status        
    else:
        status = 0
        return s, status

In [2]:
def get_two_area_stdev(gray_image):

    
    cropped = gray_image[0:16, 0:32]
    crop2 = gray_image[16:24, 0:32]

    arr = np.array(cropped)
    arr2 = arr.flatten()

    arr_b = np.array(crop2)
    arr_b2 = arr_b.flatten()

    a = np.std(arr2, ddof=1)
    b = np.std(arr_b2, ddof=1)

    return a,b

In [3]:
def magic_eraser(img, threshold):
    img_copy = img
    shape = img.shape
    ymax = shape[0]
    xmax = shape[1]
    for i in range(ymax):   
        for j in range(xmax):
            count = 0
            adjacent_list = []  

            now_pixel = int(img[i][j])

            if j-1 >= 0:
                count+=1
                up_pixel = int(img[i][j-1])

                if abs(up_pixel - now_pixel) > threshold:               
                    adjacent_list.append(up_pixel)            

            if j+1 < xmax:
                count+=1
                down_pixel = int(img[i][j+1])

                if abs(down_pixel - now_pixel) > threshold:               
                    adjacent_list.append(down_pixel)   

            if i-1 >= 0:
                count+=1
                left_pixel = int(img[i-1][j])

                if abs(left_pixel - now_pixel) > threshold:               
                    adjacent_list.append(left_pixel) 

            if i+1 < ymax:
                count+=1
                right_pixel = int(img[i+1][j])

                if abs(right_pixel - now_pixel) > threshold:  
                    adjacent_list.append(right_pixel)

            if count == len(adjacent_list):

                average = int(sum(adjacent_list)/count)          
                img_copy[i][j] = average 
        
    return img_copy

In [4]:
def list_to_stack(list_a, restrict):
    if len(list_a)>restrict:
        list_a.pop(0)   

In [5]:
def calculate_slope(x1,y1,x2,y2):
    x = (y2 - y1) / (x2 - x1)
    return x

In [6]:
def heat_remain_algorithm(b):
    slope = calculate_slope(0,b[0],1,b[2])
    print("slope:",slope)
    if slope <= -3:
        return "leave bed"
    else:
        return "still on bed"

In [ ]:
def heat_remain_algorithm(b):
    slope = calculate_slope(0,b[0],2,b[2])
    if slope <= -1.5 and b[1]<=b[0] and b[1]>=b[2]:
        return "leave bed"
    else:
        return "still on bed"
        
        
#     d1 = b[0] - b[1]
#     d2 = b[1] - b[2]
#     d3 = b[0] - b[2]
#     if d1 == 0 and d2 > 2:
#         return "leave bed"
#     elif d1 > 2 and d2 == 0:
#         return "leave bed"
#     elif d1 > 0 and d2 > 0 and d3 > 2:
#         return "leave bed"
#     else:
#         return "still on bed"

In [7]:
b = [19,17,14]
a = heat_remain_algorithm(b)
print(a)

slope: -5.0
leave bed


In [7]:
def leave_bed_algorithm(l,b):
    
    s1 = [[10,11],[0,10,11],[1,10,11],[11,10,11]]
    s2 = [[10,11,10]]
    s3 = [[10,11,0], [10,1,0], [11, 1, 0], [1,11,0]]
    
    if l in s1:
        ans = "notice"  
            
    elif l in s2:
        
        final = heat_remain_algorithm(b)   
        
        if final == "leave bed":
            ans = "leave bed"
            l = [l[-1]]
            b = [b[-1]]            
        else:
            ans = "still on bed"
            l = [l[-1]]
            b = [b[-1]]
        
    elif l in s3:
        ans = "leave bed"
        l = [l[-1]]
        b = [b[-1]]        
    else:
        ans = "do not care"
        
    return ans,l,b

In [8]:
def leave_bed_algorithm(l,b):
    
    situation1 = [[10,11,10]]
    situation2 = [[10,11,0], [10,1,0], [11, 1, 0], [1,11,0]]
    
    if l in situation1:
        
        final = heat_remain_algorithm(b)   
        
        if final == "leave bed":
            ans = "leave bed"
            l = [l[-1]]
            b = [b[-1]]
            
        else:
            ans = "still on bed"
            l = [l[-1]]
            b = [b[-1]]
            
    elif l in situation2:
        ans = "leave bed"
        l = [l[-1]]
        b = [b[-1]]
        
    else:
        ans = "do not care"
        
    return ans,l,b
    

In [ ]:
def leave_bed_algorithm(l,b):
    
    if l == [10,11,10] or l == [10,1,10]:
        
        final = heat_remain_algorithm(b)
    
        if final == "leave bed":
            ans = "leave bed"
            l = [l[-1]]
            b = [b[-1]]
            return ans, l, b
        else:
            ans = "still on bed"
            l = [l[-1]]
            b = [b[-1]]
            return ans, l, b
            
    elif l == [10,11,0] or l == [10,1,0] or l == [11, 1, 0] or l == [1,11,0]:
        ans = "leave bed"
        l = [l[-1]]
        b = [b[-1]]
        return ans, l, b
    
#     elif l == [0,11,10] or l == [0,1,10] or l == [1,11,10]:
#         ans = "enter bed"
#         l = [l[-1]]
#         b = [b[-1]]
#         return ans, l, b
        
    else:
        ans = "do not care"
        return ans,l,b

## start

In [8]:
model_path = r"C:\Users\ee527\Dementia_Vision_beta\model\test.h5"
model = tf.keras.models.load_model(model_path)

In [10]:
directory_name = r"D:\Dementia_Data_Preprocessing\testing_data\20211012\out_image"

In [12]:
pre = 99
l = []
b = []
find = []
restrict=3
ans = "initial"
padding = 0
for file_name in os.listdir(directory_name):
    #time1 = time.time() 
    print(file_name) 
    
    image_path = directory_name + "/" + file_name  
    raw = cv2.imread(image_path,0)
    
    bed_dev, floor_dev = get_two_area_stdev(raw)
    bed_dev = int(bed_dev)
    floor_dev = int(floor_dev)
           
        
    # 圖片前處理
    process1 = magic_eraser(raw, threshold=15)
    process1_copy = process1.copy()
    process1[0:16, 0:32] = padding
    process1_copy[16:24, 0:32] = padding 
    
    
    #cv2.imwrite( r"D:\test_leave_bed\1012b" + "/" + file_name, process1_copy) 
    #cv2.imwrite( r"D:\test_error\1011f" + "/" + file_name, process1) 
   
    # 開始預測
    image1 = load(process1_copy)
    image2 = load(process1)
    
    score1, result1 = predict_result(image1,threshold=0.95)
    score2, result2 = predict_result(image2,threshold=0.9)
    
    
    # 預測結果: 0,1,10,11
    final = result1 * 10 + result2
    #print(final)
    
#     if pre != final:
#         l.append(final)
#     b.append(bed_dev)
    
   
    
    if ans == "notice":
        if pre != final:
            l.append(final)
            b.append(bed_dev)
    else:
        # 狀態變化才紀錄
        if pre != final:
            l.append(final)
        b.append(bed_dev)  
        
    
    list_to_stack(l, restrict)
    list_to_stack(b, restrict)   
    print(l,b)
    
    ans,l,b = leave_bed_algorithm(l,b)       
    print(ans)
    if ans == "leave bed":
        find.append(file_name)
          
    pre = final
    
#     time2 = time.time()
#     print(time2-time1)
    print("")

20211012_t000001.jpg
[0] [6]
do not care

20211012_t000003.jpg
[0] [6, 6]
do not care

20211012_t000005.jpg
[0] [6, 6, 7]
do not care

20211012_t000007.jpg
[0] [6, 7, 6]
do not care

20211012_t000009.jpg
[0] [7, 6, 6]
do not care

20211012_t000011.jpg
[0] [6, 6, 6]
do not care

20211012_t000013.jpg
[0] [6, 6, 6]
do not care

20211012_t000015.jpg
[0] [6, 6, 6]
do not care

20211012_t000017.jpg
[0] [6, 6, 6]
do not care

20211012_t000019.jpg
[0] [6, 6, 6]
do not care

20211012_t000021.jpg
[0] [6, 6, 6]
do not care

20211012_t000023.jpg
[0] [6, 6, 6]
do not care

20211012_t000025.jpg
[0] [6, 6, 6]
do not care

20211012_t000027.jpg
[0] [6, 6, 6]
do not care

20211012_t000029.jpg
[0] [6, 6, 6]
do not care

20211012_t000031.jpg
[0] [6, 6, 6]
do not care

20211012_t000033.jpg
[0] [6, 6, 6]
do not care

20211012_t000035.jpg
[0] [6, 6, 5]
do not care

20211012_t000037.jpg
[0] [6, 5, 6]
do not care

20211012_t000039.jpg
[0] [5, 6, 6]
do not care

20211012_t000041.jpg
[0] [6, 6, 6]
do not care

2

[0] [6, 6, 6]
do not care

20211012_t000547.jpg
[0] [6, 6, 6]
do not care

20211012_t000549.jpg
[0] [6, 6, 6]
do not care

20211012_t000551.jpg
[0] [6, 6, 6]
do not care

20211012_t000553.jpg
[0] [6, 6, 6]
do not care

20211012_t000555.jpg
[0] [6, 6, 6]
do not care

20211012_t000557.jpg
[0] [6, 6, 5]
do not care

20211012_t000559.jpg
[0] [6, 5, 6]
do not care

20211012_t000601.jpg
[0] [5, 6, 5]
do not care

20211012_t000603.jpg
[0] [6, 5, 6]
do not care

20211012_t000605.jpg
[0] [5, 6, 6]
do not care

20211012_t000607.jpg
[0] [6, 6, 6]
do not care

20211012_t000609.jpg
[0] [6, 6, 6]
do not care

20211012_t000611.jpg
[0] [6, 6, 6]
do not care

20211012_t000613.jpg
[0] [6, 6, 6]
do not care

20211012_t000615.jpg
[0] [6, 6, 8]
do not care

20211012_t000617.jpg
[0] [6, 8, 6]
do not care

20211012_t000619.jpg
[0] [8, 6, 6]
do not care

20211012_t000621.jpg
[0] [6, 6, 6]
do not care

20211012_t000623.jpg
[0] [6, 6, 6]
do not care

20211012_t000625.jpg
[0] [6, 6, 6]
do not care

20211012_t000

[0] [5, 5, 6]
do not care

20211012_t001129.jpg
[0] [5, 6, 5]
do not care

20211012_t001131.jpg
[0] [6, 5, 6]
do not care

20211012_t001133.jpg
[0] [5, 6, 6]
do not care

20211012_t001135.jpg
[0] [6, 6, 6]
do not care

20211012_t001137.jpg
[0] [6, 6, 6]
do not care

20211012_t001139.jpg
[0] [6, 6, 6]
do not care

20211012_t001141.jpg
[0] [6, 6, 6]
do not care

20211012_t001143.jpg
[0] [6, 6, 6]
do not care

20211012_t001145.jpg
[0] [6, 6, 5]
do not care

20211012_t001147.jpg
[0] [6, 5, 6]
do not care

20211012_t001149.jpg
[0] [5, 6, 6]
do not care

20211012_t001151.jpg
[0] [6, 6, 6]
do not care

20211012_t001153.jpg
[0] [6, 6, 5]
do not care

20211012_t001155.jpg
[0] [6, 5, 6]
do not care

20211012_t001157.jpg
[0] [5, 6, 6]
do not care

20211012_t001159.jpg
[0] [6, 6, 6]
do not care

20211012_t001201.jpg
[0] [6, 6, 6]
do not care

20211012_t001203.jpg
[0] [6, 6, 5]
do not care

20211012_t001205.jpg
[0] [6, 5, 6]
do not care

20211012_t001207.jpg
[0] [5, 6, 6]
do not care

20211012_t001

[0] [6, 6, 5]
do not care

20211012_t001713.jpg
[0] [6, 5, 6]
do not care

20211012_t001715.jpg
[0] [5, 6, 6]
do not care

20211012_t001717.jpg
[0] [6, 6, 6]
do not care

20211012_t001719.jpg
[0] [6, 6, 6]
do not care

20211012_t001721.jpg
[0] [6, 6, 6]
do not care

20211012_t001723.jpg
[0] [6, 6, 6]
do not care

20211012_t001725.jpg
[0] [6, 6, 6]
do not care

20211012_t001727.jpg
[0] [6, 6, 6]
do not care

20211012_t001729.jpg
[0] [6, 6, 6]
do not care

20211012_t001731.jpg
[0] [6, 6, 7]
do not care

20211012_t001733.jpg
[0] [6, 7, 6]
do not care

20211012_t001735.jpg
[0] [7, 6, 7]
do not care

20211012_t001737.jpg
[0] [6, 7, 6]
do not care

20211012_t001739.jpg
[0] [7, 6, 6]
do not care

20211012_t001741.jpg
[0] [6, 6, 6]
do not care

20211012_t001743.jpg
[0] [6, 6, 5]
do not care

20211012_t001745.jpg
[0] [6, 5, 6]
do not care

20211012_t001747.jpg
[0] [5, 6, 6]
do not care

20211012_t001749.jpg
[0] [6, 6, 6]
do not care

20211012_t001751.jpg
[0] [6, 6, 5]
do not care

20211012_t001

[0] [5, 6, 6]
do not care

20211012_t002257.jpg
[0] [6, 6, 6]
do not care

20211012_t002259.jpg
[0] [6, 6, 5]
do not care

20211012_t002301.jpg
[0] [6, 5, 6]
do not care

20211012_t002303.jpg
[0] [5, 6, 6]
do not care

20211012_t002305.jpg
[0] [6, 6, 6]
do not care

20211012_t002307.jpg
[0] [6, 6, 6]
do not care

20211012_t002309.jpg
[0] [6, 6, 6]
do not care

20211012_t002311.jpg
[0] [6, 6, 6]
do not care

20211012_t002313.jpg
[0] [6, 6, 6]
do not care

20211012_t002315.jpg
[0] [6, 6, 6]
do not care

20211012_t002317.jpg
[0] [6, 6, 6]
do not care

20211012_t002319.jpg
[0] [6, 6, 6]
do not care

20211012_t002321.jpg
[0] [6, 6, 6]
do not care

20211012_t002323.jpg
[0] [6, 6, 6]
do not care

20211012_t002325.jpg
[0] [6, 6, 6]
do not care

20211012_t002327.jpg
[0] [6, 6, 6]
do not care

20211012_t002329.jpg
[0] [6, 6, 5]
do not care

20211012_t002331.jpg
[0] [6, 5, 6]
do not care

20211012_t002333.jpg
[0] [5, 6, 6]
do not care

20211012_t002335.jpg
[0] [6, 6, 6]
do not care

20211012_t002

...

[0, 1, 0] [6, 6, 6]
do not care

20211012_t233745.jpg
[0, 1, 0] [6, 6, 5]
do not care

20211012_t233747.jpg
[0, 1, 0] [6, 5, 6]
do not care

20211012_t233749.jpg
[0, 1, 0] [5, 6, 6]
do not care

20211012_t233751.jpg
[0, 1, 0] [6, 6, 6]
do not care

20211012_t233753.jpg
[0, 1, 0] [6, 6, 6]
do not care

20211012_t233755.jpg
[0, 1, 0] [6, 6, 5]
do not care

20211012_t233757.jpg
[0, 1, 0] [6, 5, 5]
do not care

20211012_t233759.jpg
[0, 1, 0] [5, 5, 6]
do not care

20211012_t233801.jpg
[0, 1, 0] [5, 6, 6]
do not care

20211012_t233803.jpg
[0, 1, 0] [6, 6, 6]
do not care

20211012_t233805.jpg
[0, 1, 0] [6, 6, 6]
do not care

20211012_t233807.jpg
[0, 1, 0] [6, 6, 5]
do not care

20211012_t233809.jpg
[0, 1, 0] [6, 5, 6]
do not care

20211012_t233811.jpg
[0, 1, 0] [5, 6, 6]
do not care

20211012_t233815.jpg
[0, 1, 0] [6, 6, 5]
do not care

20211012_t233817.jpg
[0, 1, 0] [6, 5, 6]
do not care

20211012_t233819.jpg
[0, 1, 0] [5, 6, 6]
do not care

20211012_t233821.jpg
[0, 1, 0] [6, 6, 6]
do not c

[0, 1, 0] [5, 5, 6]
do not care

20211012_t234251.jpg
[0, 1, 0] [5, 6, 6]
do not care

20211012_t234253.jpg
[0, 1, 0] [6, 6, 6]
do not care

20211012_t234255.jpg
[0, 1, 0] [6, 6, 6]
do not care

20211012_t234257.jpg
[0, 1, 0] [6, 6, 6]
do not care

20211012_t234259.jpg
[0, 1, 0] [6, 6, 6]
do not care

20211012_t234301.jpg
[0, 1, 0] [6, 6, 6]
do not care

20211012_t234303.jpg
[0, 1, 0] [6, 6, 6]
do not care

20211012_t234305.jpg
[0, 1, 0] [6, 6, 6]
do not care

20211012_t234307.jpg
[0, 1, 0] [6, 6, 6]
do not care

20211012_t234309.jpg
[0, 1, 0] [6, 6, 5]
do not care

20211012_t234311.jpg
[0, 1, 0] [6, 5, 6]
do not care

20211012_t234313.jpg
[0, 1, 0] [5, 6, 6]
do not care

20211012_t234315.jpg
[0, 1, 0] [6, 6, 6]
do not care

20211012_t234317.jpg
[0, 1, 0] [6, 6, 6]
do not care

20211012_t234319.jpg
[0, 1, 0] [6, 6, 6]
do not care

20211012_t234321.jpg
[0, 1, 0] [6, 6, 6]
do not care

20211012_t234323.jpg
[0, 1, 0] [6, 6, 5]
do not care

20211012_t234325.jpg
[0, 1, 0] [6, 5, 6]
do not c

[0, 1, 0] [6, 6, 6]
do not care

20211012_t234759.jpg
[0, 1, 0] [6, 6, 6]
do not care

20211012_t234801.jpg
[0, 1, 0] [6, 6, 6]
do not care

20211012_t234803.jpg
[0, 1, 0] [6, 6, 6]
do not care

20211012_t234805.jpg
[0, 1, 0] [6, 6, 6]
do not care

20211012_t234807.jpg
[0, 1, 0] [6, 6, 6]
do not care

20211012_t234809.jpg
[0, 1, 0] [6, 6, 6]
do not care

20211012_t234811.jpg
[0, 1, 0] [6, 6, 7]
do not care

20211012_t234813.jpg
[0, 1, 0] [6, 7, 6]
do not care

20211012_t234815.jpg
[0, 1, 0] [7, 6, 6]
do not care

20211012_t234817.jpg
[0, 1, 0] [6, 6, 6]
do not care

20211012_t234819.jpg
[0, 1, 0] [6, 6, 7]
do not care

20211012_t234821.jpg
[0, 1, 0] [6, 7, 6]
do not care

20211012_t234823.jpg
[0, 1, 0] [7, 6, 6]
do not care

20211012_t234825.jpg
[0, 1, 0] [6, 6, 6]
do not care

20211012_t234827.jpg
[0, 1, 0] [6, 6, 6]
do not care

20211012_t234829.jpg
[0, 1, 0] [6, 6, 6]
do not care

20211012_t234831.jpg
[0, 1, 0] [6, 6, 6]
do not care

20211012_t234833.jpg
[0, 1, 0] [6, 6, 6]
do not c

[0, 1, 0] [7, 6, 6]
do not care

20211012_t235305.jpg
[0, 1, 0] [6, 6, 6]
do not care

20211012_t235307.jpg
[0, 1, 0] [6, 6, 7]
do not care

20211012_t235309.jpg
[0, 1, 0] [6, 7, 6]
do not care

20211012_t235311.jpg
[0, 1, 0] [7, 6, 5]
do not care

20211012_t235313.jpg
[0, 1, 0] [6, 5, 5]
do not care

20211012_t235315.jpg
[0, 1, 0] [5, 5, 6]
do not care

20211012_t235317.jpg
[0, 1, 0] [5, 6, 6]
do not care

20211012_t235319.jpg
[0, 1, 0] [6, 6, 5]
do not care

20211012_t235321.jpg
[0, 1, 0] [6, 5, 6]
do not care

20211012_t235323.jpg
[0, 1, 0] [5, 6, 6]
do not care

20211012_t235325.jpg
[0, 1, 0] [6, 6, 7]
do not care

20211012_t235327.jpg
[0, 1, 0] [6, 7, 7]
do not care

20211012_t235329.jpg
[0, 1, 0] [7, 7, 6]
do not care

20211012_t235331.jpg
[0, 1, 0] [7, 6, 7]
do not care

20211012_t235333.jpg
[0, 1, 0] [6, 7, 6]
do not care

20211012_t235335.jpg
[0, 1, 0] [7, 6, 6]
do not care

20211012_t235337.jpg
[0, 1, 0] [6, 6, 6]
do not care

20211012_t235339.jpg
[0, 1, 0] [6, 6, 8]
do not c

[0, 1, 0] [7, 6, 7]
do not care

20211012_t235813.jpg
[0, 1, 0] [6, 7, 5]
do not care

20211012_t235815.jpg
[0, 1, 0] [7, 5, 6]
do not care

20211012_t235817.jpg
[0, 1, 0] [5, 6, 6]
do not care

20211012_t235819.jpg
[0, 1, 0] [6, 6, 6]
do not care

20211012_t235821.jpg
[0, 1, 0] [6, 6, 6]
do not care

20211012_t235823.jpg
[0, 1, 0] [6, 6, 6]
do not care

20211012_t235825.jpg
[0, 1, 0] [6, 6, 6]
do not care

20211012_t235827.jpg
[0, 1, 0] [6, 6, 6]
do not care

20211012_t235829.jpg
[0, 1, 0] [6, 6, 6]
do not care

20211012_t235831.jpg
[0, 1, 0] [6, 6, 6]
do not care

20211012_t235833.jpg
[0, 1, 0] [6, 6, 6]
do not care

20211012_t235835.jpg
[0, 1, 0] [6, 6, 7]
do not care

20211012_t235837.jpg
[0, 1, 0] [6, 7, 6]
do not care

20211012_t235839.jpg
[0, 1, 0] [7, 6, 6]
do not care

20211012_t235841.jpg
[0, 1, 0] [6, 6, 6]
do not care

20211012_t235843.jpg
[0, 1, 0] [6, 6, 5]
do not care

20211012_t235845.jpg
[0, 1, 0] [6, 5, 6]
do not care

20211012_t235847.jpg
[0, 1, 0] [5, 6, 6]
do not c

In [13]:
print(find)

['20211012_t175951.jpg', '20211012_t180139.jpg', '20211012_t180243.jpg', '20211012_t180327.jpg', '20211012_t180347.jpg', '20211012_t180411.jpg', '20211012_t180445.jpg', '20211012_t180457.jpg', '20211012_t180545.jpg', '20211012_t180935.jpg', '20211012_t181035.jpg', '20211012_t181139.jpg']


## test

In [ ]:
directory_name = r"D:\test9"


pre = 99
l = []
b = []
find = []
system = "close"
for file_name in os.listdir(directory_name):
    print(file_name)
  
    image_path = directory_name + "/" + file_name  
    raw = cv2.imread(image_path,0)
    
    bed_dev, floor_dev = get_two_area_stdev(raw)
    bed_dev = int(bed_dev)
    floor_dev = int(floor_dev)
    
    ################################################
#     plot_bed.append(bed_dev)
#     plot_floor.append(floor_dev)
    ################################################
       
        
    # 圖片前處理
    process1 = magic_eraser(raw, threshold=15)
    process1_copy = process1.copy()
    process1[0:16, 0:32] = 0
    process1_copy[16:24, 0:32] = 0    
    
#     cv2.imwrite( r"D:\test99\b" + "/" + file_name, process1_copy) 
#     cv2.imwrite( r"D:\test99\f" + "/" + file_name, process1) 
   
    # 開始預測
    image1 = load(process1_copy)
    image2 = load(process1)
    score1, result1 = predict_result(image1,threshold=0.95)
    score2, result2 = predict_result(image2,threshold=0.85)
    print(score1,score2)
    ########################################
#     mm = magic_eraser(raw, threshold=15)
#     aaa = load(mm)
#     score, result = predict_result(aaa)
#     plot_old.append(result)
    ########################################
    
    # 預測結果: 0,1,10,11
    final = result1 * 10 + result2
    #print(final)
     
    # 狀態變化才紀錄
    if pre != final:
        l.append(final)
        
    b.append(bed_dev)

    list_to_stack(l, restrict=3)
    list_to_stack(b, restrict=3)   
    print(l,b)
    
    ans,l,b = leave_bed_algorithm(l,b)  
    
#     if system == "close":
#         if ans == "enter bed":
#             print(ans)
#             system = "active"
    
#     if system == "active":
        
    if ans == "leave bed":
        print("leave bed")
        print("")
        find.append(file_name)
    elif ans == "still on bed":
        print("still on bed")
        print("")
    else:
        print("")
  
        
    print("")      
    pre = final

In [ ]:
print(find)

## test2

In [ ]:

raw = cv2.imread(r"D:\Dementia_Data_Preprocessing\testing_data\20211007-1011\out_image\20211007_t220232.jpg",0)

bed_dev, floor_dev = get_two_area_stdev(raw)
bed_dev = int(bed_dev)
floor_dev = int(floor_dev)
           

# 圖片前處理
process1 = magic_eraser(raw, threshold=15)
process1_copy = process1.copy()
process1[0:16, 0:32] = 0
process1_copy[16:24, 0:32] = 0   
cv2.imwrite( r"D:\a.jpg", process1_copy) 
cv2.imwrite( r"D:\b.jpg" ,process1) 

# 開始預測
image1 = load(process1_copy)
image2 = load(process1)
score1, result1 = predict_result(image1,threshold=0.95)
score2, result2 = predict_result(image2,threshold=0.85)
print(bed_dev,floor_dev)
print(score1,score2)
